#Get started immediately with your Data with AI Functions

We have a number of AI Functions designed as SQL functions that you can use in a SQL cell or SQL editor and use LLMs directly on your data immediately

1. ai_analyze_sentiment
2. ai_classify
3. ai_extract
4. ai_fix_grammar
5. ai_gen
6. ai_mask
7. ai_similarity
8. ai_summarize
9. ai_translate
10. ai_query
11. ai_parse_document

We will run a demo of a few of these functions below. 




In [0]:
%run ./config

In [0]:
%python
dbutils.widgets.text("catalog_name", catalog_name)
dbutils.widgets.text("schema_name", schema_name)

### Demo Overview
For today's demo, we’re using a telecom dataset with the ultimate goal of creating a customer representative agent. One of the first steps in building any agent is data preparation. 
In this notebook, we will see how to leverage AI functions to clean, extract, and turn unstructured data into readily usable data by our agents.

### ai_query
The ai_query() function allows you to query machine learning models and large language models served using Mosaic AI Model Serving. To do so, this function invokes an existing Mosaic AI Model Serving endpoint and parses and returns its response. Databricks recommends using ai_query with Model Serving for batch inference. We can switch models depending on what we are trying to do. 

Documentation: https://docs.databricks.com/en/large-language-models/ai-functions.html#ai_query

In this example, we'll assign priority to support tickets to help teams/agents take next steps.


In [0]:
%sql 
SELECT
  `description`,   -- Placeholder for the input column
  ai_query(
    'databricks-meta-llama-3-1-8b-instruct', -- 'databricks-meta-llama-3-3-70b-instruct'
    CONCAT(format_string('Analyze the support ticket and assign priority. The output must only be one word from this list: [low, medium, high, critical]. Do not add any extra text. Ticket: '), `description`)    -- Placeholder for the prompt and input
  ) AS ai_priority  -- Placeholder for the output column
FROM identifier(:catalog_name||'.'||:schema_name||'.'||'support_tickets')
limit 10;


## ai_parse_document

The ai_parse_document() function invokes a state-of-the-art generative AI model from Databricks Foundation Model APIs to extract structured content from unstructured documents. The following file formats are supported:

- PDF
- JPG / JPEG
- PNG
- DOC/DOCX
- PPT/PPTX

Documentation: https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_parse_document 

Let's see an example extracting info from a sample customer invoice:

In [0]:
WITH parsed_invoice AS (
  SELECT
    path,
    content
  FROM
    READ_FILES('/Volumes/'||:catalog_name||'/'||:schema_name||'/invoice', format => 'binaryFile')
)
SELECT
  path,
  ai_parse_document(content) AS parsed
FROM
  parsed_invoice

Let's parse the extracted content further to create structured data to use for downstream applications.

In [0]:
-- Parse raw data from invoice file
WITH parsed_invoice AS (
  SELECT
    path,
    content
  FROM
    READ_FILES('/Volumes/'||:catalog_name||'/'||:schema_name||'/invoice', format => 'binaryFile')
),
raw_data AS (
  SELECT
    path,
    ai_parse_document(content) AS parsed
  FROM
    parsed_invoice
),
-- Step 2: Extract all text content from the parsed document
extracted_content AS (
  SELECT
    path,
    -- Concatenate all text elements into full document text
    concat_ws(
      '\n\n',
      transform(
        filter(
          try_cast(parsed:document:elements AS ARRAY<VARIANT>),
          element -> try_cast(element:type AS STRING) IN ('text', 'section_header', 'title')
        ),
        element -> try_cast(element:content AS STRING)
      )
    ) AS full_text,
    -- Extract tables separately (pricing, line items)
    transform(
      filter(
        try_cast(parsed:document:elements AS ARRAY<VARIANT>),
        element -> try_cast(element:type AS STRING) = 'table'
      ),
      element -> try_cast(element:content AS STRING)
    ) AS tables,
    -- Get page count
    try_cast(parsed:document:page_count AS INT) AS page_count,
    -- Check for errors
    try_cast(parsed:error_status AS STRING) AS parse_error
  FROM raw_data
)

Select 
  ai_extract(full_text, 
  array('Customer name', 'Customer address', 'Invoice number', 'Invoice date', 'Total amount')) as extracted_fields,
  ai_query('databricks-meta-llama-3-1-8b-instruct', 
  CONCAT(format_string('Summarize the transaction details in 1 sentence. Transaction details: '), `full_text`)) as summary,
  full_text, 
  tables
from extracted_content


### Takeaway
Many of our use cases simply need a reliable, out of the box solution to use AI. AI functions enable this for our customers and ai_query helps scale workloads. For more information on how to incorporate AI functions into your data engineering workflows, check out [Deploy batch inference pipelines](https://docs.databricks.com/aws/en/large-language-models/batch-inference-pipelines).  